# Reti Domotica Anomaly Detection - Team 1

## Componenti del Team

- Francesco CATANIA
- Giorgio DEIANA
- Marco SASSO

## Setup dell'ambiente

1. Installare *ODBC Driver for SQL Server* (https://docs.microsoft.com/it-it/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15), necessario per l'interazione col database Azure SQL;


2. Da Visual Studio Code cliccare *File -> Open Folder* e selezionare una cartella che svolga il ruolo di repository locale dei file del progetto;


3. Aprire il terminale cliccando su *Terminal -> New Terminal* ed effettuare il clone della repository GitHub del progetto digitando il comando ***git clone https://github.com/degjoster/RetiDomoticaAnomalyDetect.git"***;


4. Creare un nuovo ambiente Python digitando sul terminale il comando ***virtualenv .venv***;


5. Attivare l'ambiente creato al punto 4 digitando sul terminale il comando ***.venv\Scripts\activate***


6. Spostarsi all'interno della cartella *RetiDomoticaAnomalyDetect* digitando da terminale il comando ***cd RetiDomoticaAnomalyDetect***; 


7. Installare le librerie indicate nel file *requirements.txt* digitando da terminale il comando ***pip install -r "requirements.txt"***;


8. (**Facoltativo**) All'interno del file "settings.json" inserire le seguenti stringhe: "python.pythonPath": "{NOMEAMBIENTE}\\Scripts\\python.exe",
    "python.linting.pylintPath": "{NOMEAMBIENTE}\\bin\\pylint"

## Note Metodologiche

#### Modello adottato e addestramento

L'algoritmo di Machine Learning utilizzato per la rilevazione delle anomalie è l'Isolation Forest implementato all'interno della libreria Scikit-Learn. Si è scelto di addestrare cinque differenti modelli di Machine Learning - uno per ciascuna unità di misura rilevata ('ppm', '%', 'C°', 'W', 'Wh') - al fine di separare opportunamente i processi di ricerca di eventuali anomalie a seconda delle tipologie di rilevazioni presenti nel dataset "log". Dal progetto sono state esclusi i dati di tipo booleano riguardanti la rilevazione delle presenze.

I modelli sopracitati sono stati addestrati eseguendo un apposito notebook Python all'interno della piattaforma Databricks di Microsoft Azure, consultabile nella directory *Notebooks* della repository GitHub. I dati per l'addestramento sono stati estratti dal database SQL *CampusData* presente tra le risorse della sottoscrizione Microsoft Azure del corso.

Per il training di ciascun modello sono state considerate features in grado di cogliere il valore, la tipologia e l'eventuale stagionalità delle rilevazioni effettuate dai sensori del Campus, nello specifico:

- mese della rilevazione (derivata dal campo *Data* della tabella *dbo.Log* del database SQL)
- giorno della settimana della rilevazione (derivata dal campo *Data* del dataset *dbo.Log* del database SQL)
- valore della rilevazione (feature *Value* della tabella *dbo.Log* del database SQL)
- tipologia di rilevazione (feature *Id* della tabella *dbo.Group* del database SQL, ottenuta tramite join con la tabella *dbo.Log*)

#### Applicazione Streamlit

L'utente può interagire con le funzionalità implementate nel progetto attraverso un'applicazione realizzata in Streamlit. Per accedere all'applicazione:

- da terminale, digitare ***cd RetiDomoticaAnomalyDetect\DomoticaStreamLit*** per posizionarsi all'interno della directory *DomoticaStreamLit*
- sempre da terminale, digitare ***streamlit run app.py*** per lanciare l'applicazione


L'applicazione si presenta come una pagina web composta da due componenti principali: un menu centrale dal quale è possibile richiamare le funzioni implementate e una side bar per la definizione dei parametri da parte dell'utente.  

All'interno dell'applicazione l'utente può svolgere due diverse attività:

1. **Caricare nuovi dati dal database SQL Azure** - diversi da quelli utilizzati per l'addestramento dei modelli - per effettuare delle nuove predizioni volte ad individuare eventuali anomalie. In questo caso l'utente deve specificare, dalla side bar, uno degli edifici del Campus Reti e la tipologia (descrizione) di rilevazione. Definiti i parametri, dal corpo centrale della pagina è possibile scegliere tra:
    - ***Carica Dati da DB scegliendo Edificio e Descrizione***, che carica i dati contenuti nella tabella *dbo.log_newData* del database, richiama l'apposito modello in base all'unità di misura e mostra in un grafico la serie storica delle rilevazioni evidenziando in rosso la presenza di eventuali anomalie;
    - ***Carica Dati da DB di "Edificio e Descrizione" e salva predizione su DB***, che non fornisce un output di tipo grafico ma salva le predizioni in formato JSON nel container *predictcontainerdomoticaadsqldb* dell'Azure Storage Account *storageaccountprogteam1*;


2. **Simulare una rilevazione di un sensore** indicando, oltre ai parametri richiesti al punto 1, la data e il valore numerico della rilevazione per determinare la presenza di un'eventuale anomalia. Analogamente al punto precedente, dal corpo centrale della pagina è possibile scegliere tra:
    - ***Effettua predizione*** che richiama l'apposito modello in base all'unità di misura della rilevazione inserita dall'utente e indica, tramite una stringa di testo, se si tratta un'anomalia;
    - ***Effettua predizione e salva su azure*** che salva la predizione in formato JSON nel container *predictcontainerdomoticaaduser* dell'Azure Storage Account *storageaccountprogteam1*.

## Navigazione cartelle della repository GitHub

Le varie componenti del progetto sono organizzate secondo la seguente struttura di cartelle:

- **Datasets** contiene i file *building.csv*, *group.csv* e *log2.csv* messi a disposizione a inizio progetto per le analisi preliminari in locale;


- **DomoticaStreamLit** contiene il file Python *app.py* per il lancio dell'applicazione web in Streamlit; 


- **Notebooks** contiene i notebook utilizzati per l'esplorazione preliminare del dataset e per il training dei modelli in Databricks; 


- **TeamLibraries** contiene le funzioni Python sviluppate internamente dal team, suddivise in tre diversi moduli:
    - *LoadData.py* racchiude le funzioni per la connessione al database SQL Azure, lo scarico dei dati e il caricamento dei modelli versionati in Databricks;
    
    - *ModelFunctions.py* comprende le funzioni per il preprocessing dei dati e la predizione tramite i modelli;
    
    - *OperationsStorageAccount.py* contiene le funzioni per la connessione allo Storage Account Azure e il salvataggio delle predizioni in formato JSON negli appositi container;
    
    ***A causa di imprevisti tecnici che si sono verificati con il serving dei modelli in Databricks, la cartella TeamLibraries contiene un backup di tutti i modelli in formato Pickle addestrati in locale***.
